In [ ]:
#@markdown ⬅ 點擊執行按鈕開始刪除指定資料夾下的個人檔案與空資料夾
#@markdown ---
#@markdown <br><br>
#@markdown # 使用者設定
#@markdown ---
#@title 刪除指定資料夾下的個人檔案與空資料夾 { display-mode: "form" }
目標資料夾 = "" #@param {type:"string", placeholder:"輸入要處理的資料夾網址"}
#@markdown - 將處理此資料夾 **內部** 的項目。
#@markdown - 請輸入瀏覽器網址列顯示的網址。
#@markdown - 你必須至少是該資料夾的 **編輯者** 才能刪除其中的項目。
#@markdown - 腳本 **只會** 刪除資料夾內 **屬於您自己** 的檔案。
#@markdown - 腳本 **只會** 刪除資料夾內 **屬於您自己且已變空** 的子資料夾。
#@markdown ---
請求間隔 = 0.2 # @param {type:"slider", min:0.1, max:5, step:0.1}
#@markdown - API 請求之間的延遲（秒），以避免觸發速率限制。
#@markdown ---
自動清除垃圾桶 = False # @param {type:"boolean"}
#@markdown - 是否在處理完畢後自動清除垃圾桶？
#@markdown ---
#@markdown <br><br>
操作確認 = "" #@param {type:"string", placeholder:"輸入您的 email 以確認操作"}
#@markdown <br><br>
# -*- coding: utf-8 -*-

# 變數名稱轉換
TARGET_FOLDER_URL_OR_ID = 目標資料夾 # Renamed variable
DELAY_BETWEEN_REQUESTS = 請求間隔
AUTO_CLEAN = 自動清除垃圾桶
EMAIL_VERIFY = 操作確認

"""
將指定資料夾內 **屬於執行者** 的所有檔案與 **空的子資料夾** 移至垃圾桶。

此腳本會：
1. 取得目前執行者的 Email。
2. 驗證指定的 '目標資料夾'。
3. 遞迴地處理 '目標資料夾' 內的項目：
    a. 如果是 **檔案** 且 **屬於您**，則移至垃圾桶。
    b. 如果是 **子資料夾**，則先遞迴處理其內部項目。
    c. 如果處理完畢後，該 **子資料夾變空** 且 **屬於您**，則將該空資料夾移至垃圾桶。
4. **只會** 處理您擁有的項目，其他人的項目會被跳過。
5. 可選擇是否自動清空垃圾桶。

**警告：此操作會將指定資料夾內您擁有的檔案與空的子資料夾移至垃圾桶。請謹慎使用！**
"""


# --- 安裝與載入套件 ---
try:
    from google.colab import auth
    IN_COLAB = True
except ImportError:
    IN_COLAB = False
    print("⚠️ 未在 Google Colab 環境中執行。將嘗試使用應用程式預設憑證 (ADC)。")
    print("   請確保已安裝 google-auth 函式庫並設定好 ADC (gcloud auth application-default login)。")

from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.auth import default
from tqdm.notebook import tqdm # 使用 notebook 版本的 tqdm
import time
import sys
import re

# --- 全域變數 ---
user_email = ''
drive_service = None
target_folder_name_global = '' # 用於顯示
total_trashed_files = 0
total_trashed_folders = 0
total_skipped = 0
total_failed = 0

# --- 輔助函數 ---

def authenticate_and_build_service():
    """處理驗證並建立 Drive API 服務"""
    global drive_service, user_email
    SCOPES = ['https://www.googleapis.com/auth/drive']
    try:
        print("⏳ 正在請求 Google Drive 授權...")
        if IN_COLAB:
            auth.authenticate_user()
            creds, _ = default(scopes=SCOPES)
        else:
            # For local execution, use ADC
            creds, _ = default(scopes=SCOPES)

        drive_service = build('drive', 'v3', credentials=creds, cache_discovery=False)
        print("✅ 授權成功，正在建立 Drive 服務...")
        # 取得使用者資訊
        about = drive_service.about().get(fields="user").execute()
        user_email = about['user']['emailAddress']
        print(f"✅ Drive 服務建立成功，目前使用者: {user_email}")
        return True
    except Exception as e:
        print(f"❌ 驗證或建立服務失敗: {e}")
        if not IN_COLAB:
            print("   提示：在本地執行，請確保已設定 Google Cloud SDK 並執行 'gcloud auth application-default login'")
        return False

def get_item_info(item_id, fields='id, name, mimeType, trashed, capabilities, owners'):
    """取得指定 ID 項目 (檔案或資料夾) 的資訊"""
    global drive_service, DELAY_BETWEEN_REQUESTS
    try:
        time.sleep(DELAY_BETWEEN_REQUESTS)
        # Use supportsAllDrives=True for compatibility with Shared Drives if needed
        item = drive_service.files().get(fileId=item_id, fields=fields, supportsAllDrives=True).execute()
        if item.get('trashed', False):
            # print(f"  ℹ️ 項目 '{item.get('name')}' (ID: {item_id}) 已在垃圾桶中。")
            return None # Treat as non-existent for processing
        return item
    except HttpError as error:
        if error.resp.status == 404:
             print(f"\n  ❌ 找不到項目 (ID: {item_id})。")
        else:
             print(f"\n  ❌ 獲取項目資訊 (ID: {item_id}) 時發生 API 錯誤: {error}")
        return None
    except Exception as e:
        print(f"\n  ❌ 獲取項目資訊 (ID: {item_id}) 時發生非預期錯誤: {e}")
        return None

def is_folder_empty(folder_id):
    """檢查資料夾是否為空"""
    global drive_service, DELAY_BETWEEN_REQUESTS
    page_token = None
    try:
        time.sleep(DELAY_BETWEEN_REQUESTS)
        response = drive_service.files().list(
            q=f"'{folder_id}' in parents and trashed=false",
            fields='files(id)',
            pageSize=1, # We only need to know if at least one item exists
            pageToken=page_token,
            supportsAllDrives=True,
            includeItemsFromAllDrives=True
        ).execute()
        return not response.get('files') # Return True if 'files' list is empty
    except HttpError as error:
        print(f"\n  ⚠️ 檢查資料夾是否為空 (ID: {folder_id}) 時發生 API 錯誤: {error}")
        return False # Assume not empty on error to be safe
    except Exception as e:
        print(f"\n  ⚠️ 檢查資料夾是否為空 (ID: {folder_id}) 時發生非預期錯誤: {e}")
        return False # Assume not empty

def extract_folder_id_from_url(url_or_id):
    """從 Google Drive 資料夾網址或 ID 中提取 ID"""
    url_or_id = url_or_id.strip()
    # Regex for standard folder URL
    match = re.search(r'/folders/([a-zA-Z0-9_-]+)', url_or_id)
    if match:
        return match.group(1)
    # Regex for URL with 'id=' parameter
    match = re.search(r'id=([a-zA-Z0-9_-]+)', url_or_id)
    if match:
        return match.group(1)
    # Regex for Shared Drive URL
    match = re.search(r'/drive/folders/([a-zA-Z0-9_-]+)', url_or_id)
    if match:
        return match.group(1)
    # If it doesn't contain typical URL parts, assume it's an ID
    if '/' not in url_or_id and '?' not in url_or_id and '=' not in url_or_id:
         if re.match(r'^[a-zA-Z0-9_-]+$', url_or_id):
              return url_or_id
    print(f"  ⚠️ 無法從輸入 '{url_or_id}' 中識別有效的資料夾 ID。")
    return None


def trash_owned_items_recursive(folder_id, folder_name, pbar):
    """遞迴地將指定資料夾內 **屬於執行者** 的檔案與 **空的子資料夾** 移至垃圾桶"""
    global drive_service, DELAY_BETWEEN_REQUESTS, user_email
    global total_trashed_files, total_trashed_folders, total_skipped, total_failed

    page_token = None
    items_in_folder = []

    # --- 1. 列出當前資料夾所有項目 ---
    while True:
        try:
            time.sleep(DELAY_BETWEEN_REQUESTS)
            param = {
                'q': f"'{folder_id}' in parents and trashed = false",
                'fields': "nextPageToken, files(id, name, mimeType, owners)",
                'pageSize': 500, # Fetch more items per request
                'pageToken': page_token,
                'supportsAllDrives': True,
                'includeItemsFromAllDrives': True
            }
            response = drive_service.files().list(**param).execute()
            items_in_folder.extend(response.get('files', []))
            page_token = response.get('nextPageToken')
            if not page_token:
                break
        except HttpError as error:
            pbar.write(f"\n  ❌ 列出資料夾 '{folder_name}' (ID: {folder_id}) 中的項目時發生 API 錯誤: {error}")
            total_failed += 1 # Count this listing error
            return # Stop processing this folder on listing error
        except Exception as e:
            pbar.write(f"\n  ❌ 列出資料夾 '{folder_name}' (ID: {folder_id}) 中的項目時發生非預期錯誤: {e}")
            total_failed += 1
            return

    # --- 2. 處理項目 ---
    for item in items_in_folder:
        item_id = item['id']
        item_name = item.get('name', f'未命名項目 (ID: {item_id})')
        mime_type = item.get('mimeType')
        is_folder = mime_type == 'application/vnd.google-apps.folder'
        item_type = "資料夾" if is_folder else "檔案"
        item_owners = item.get('owners', [])

        # --- 檢查擁有者 ---
        is_owner = False
        owner_email = "未知"
        if item_owners:
            owner_email = item_owners[0].get('emailAddress', '未知')
            if owner_email == user_email:
                is_owner = True
        else:
            pbar.write(f"    ⚠️ 警告：無法獲取 {item_type} '{item_name}' (ID: {item_id}) 的擁有者資訊，將跳過。")

        if not is_owner:
            # pbar.write(f"    ⏭️ 跳過 {item_type} '{item_name}' (擁有者: {owner_email})")
            total_skipped += 1
            pbar.update(1) # Still update progress for skipped items
            continue # Skip to next item

        # --- 處理擁有的項目 ---
        if is_folder:
            # --- 遞迴處理子資料夾 ---
            pbar.set_description(f"進入資料夾: {item_name[:20]}...")
            trash_owned_items_recursive(item_id, item_name, pbar) # Recursive call

            # --- 檢查子資料夾是否變空並嘗試刪除 ---
            pbar.set_description(f"檢查空資料夾: {item_name[:20]}...")
            if is_folder_empty(item_id):
                try:
                    time.sleep(DELAY_BETWEEN_REQUESTS)
                    drive_service.files().update(
                        fileId=item_id,
                        body={'trashed': True},
                        supportsAllDrives=True
                    ).execute()
                    # pbar.write(f"    🗑️ 已將空的資料夾 '{item_name}' (ID: {item_id}) 移至垃圾桶。")
                    total_trashed_folders += 1
                    pbar.update(1) # Update progress for the trashed folder itself
                except HttpError as error:
                    pbar.write(f"    ❌ 將空的資料夾 '{item_name}' (ID: {item_id}) 移至垃圾桶時發生 API 錯誤: {error}")
                    total_failed += 1
                    pbar.update(1) # Update progress even if failed
                except Exception as e:
                    pbar.write(f"    ❌ 將空的資料夾 '{item_name}' (ID: {item_id}) 移至垃圾桶時發生非預期錯誤: {e}")
                    total_failed += 1
                    pbar.update(1)
            else:
                 # pbar.write(f"    ℹ️ 資料夾 '{item_name}' (ID: {item_id}) 非空或檢查失敗，不移動。")
                 # No pbar update here, it was updated during recursive calls for its content
                 pass # Folder not empty or check failed, do nothing
        else:
            # --- 處理檔案 ---
            pbar.set_description(f"處理檔案: {item_name[:25]}...")
            try:
                time.sleep(DELAY_BETWEEN_REQUESTS)
                drive_service.files().update(
                    fileId=item_id,
                    body={'trashed': True},
                    supportsAllDrives=True
                ).execute()
                # pbar.write(f"    🗑️ 已將檔案 '{item_name}' (ID: {item_id}) 移至垃圾桶。")
                total_trashed_files += 1
                pbar.update(1) # Update progress for the trashed file
            except HttpError as error:
                pbar.write(f"    ❌ 將檔案 '{item_name}' (ID: {item_id}) 移至垃圾桶時發生 API 錯誤: {error}")
                total_failed += 1
                pbar.update(1)
            except Exception as e:
                pbar.write(f"    ❌ 將檔案 '{item_name}' (ID: {item_id}) 移至垃圾桶時發生非預期錯誤: {e}")
                total_failed += 1
                pbar.update(1)


def empty_trash():
    """清空 Google Drive 垃圾桶"""
    global drive_service
    try:
        drive_service.files().emptyTrash().execute()
        print("✅ Google Drive 垃圾桶已清空。")
        return True
    except HttpError as error:
        print(f"❌ 清空垃圾桶時發生 API 錯誤: {error}")
        return False
    except Exception as e:
        print(f"❌ 清空垃圾桶時發生非預期錯誤: {e}")
        return False

# --- 主執行流程 ---

if __name__ == "__main__":
    print("🚀 開始執行刪除資料夾下個人檔案與空資料夾腳本 🚀")
    print("⚠️ 警告：此腳本將目標資料夾內 **您擁有** 的檔案與空的子資料夾移至垃圾桶！")

    # --- 1. 檢查輸入 ---
    if not TARGET_FOLDER_URL_OR_ID:
         sys.exit("❌ 請先在 '使用者設定' 中填入 '目標資料夾' 的網址或 ID。")

    target_folder_id = extract_folder_id_from_url(TARGET_FOLDER_URL_OR_ID)

    if not target_folder_id:
        sys.exit(f"❌ 無法從輸入 '{TARGET_FOLDER_URL_OR_ID}' 中提取有效的 Google Drive 資料夾 ID。請檢查輸入。")

    # --- 2. 驗證與建立服務 ---
    if not authenticate_and_build_service():
        sys.exit("❌ 無法繼續執行，請檢查驗證問題。")

    if not user_email:
         sys.exit("❌ 無法取得目前使用者 Email，無法繼續。")

    # --- 3. Email 確認 ---
    if EMAIL_VERIFY.strip() != user_email:
        print(f"\n🚫 操作已取消。輸入的 Email ('{EMAIL_VERIFY}') 與偵測到的使用者 Email ('{user_email}') 不符或未輸入。")
        sys.exit("腳本已中止。")

    # --- 4. 驗證目標資料夾並檢查權限 ---
    print(f"\n🔍 正在驗證目標資料夾 ID: {target_folder_id}...")
    target_folder_info = get_item_info(target_folder_id, fields='id, name, mimeType, capabilities')

    if not target_folder_info:
        print(f"❌ 無法驗證或存取目標資料夾 ID '{target_folder_id}'。")
        print(f"   請確認 ID 是否正確、資料夾是否存在且未被移入垃圾桶，以及您是否有權限存取。")
        sys.exit("無法繼續執行。")

    if target_folder_info.get('mimeType') != 'application/vnd.google-apps.folder':
         sys.exit(f"❌ 提供的 ID '{target_folder_id}' 不是一個資料夾。")

    target_folder_name_global = target_folder_info.get('name', '未命名資料夾')
    print(f"✅ 目標資料夾確認: '{target_folder_name_global}' (ID: {target_folder_id})")

    # 檢查是否有編輯權限 (通常需要編輯權限才能刪除子項目)
    if not target_folder_info.get('capabilities', {}).get('canEdit'):
         print(f"⚠️ 警告：您可能沒有編輯目標資料夾 '{target_folder_name_global}' 的權限。")
         print(f"   這可能導致無法將其中的項目移至垃圾桶。腳本將嘗試繼續，但可能失敗。")
         # sys.exit("權限不足，無法繼續執行。") # 或者直接退出

    # --- 5. 執行遞迴刪除 ---
    print(f"\n⏳ 正在處理資料夾 '{target_folder_name_global}' 內的項目...")
    # 預先計算總數比較困難且耗時，這裡使用未知總數的進度條
    with tqdm(desc="處理中", unit=" 項") as pbar:
        try:
            trash_owned_items_recursive(target_folder_id, target_folder_name_global, pbar)
        except KeyboardInterrupt:
             print("\n🚨 操作被使用者手動中斷 (Ctrl+C)。")
        except Exception as e:
             print(f"\n🚨 處理過程中發生嚴重非預期錯誤: {e}")
             import traceback
             traceback.print_exc()

    print("\n--- 處理結果 ---")
    print(f"🗑️ 成功移至垃圾桶的檔案: {total_trashed_files}")
    print(f"🗑️ 成功移至垃圾桶的空資料夾: {total_trashed_folders}")
    print(f"⏭️ 因權限或其他原因跳過的項目: {total_skipped}")
    print(f"❌ 處理失敗的項目/操作: {total_failed}")

    items_were_trashed = (total_trashed_files + total_trashed_folders) > 0

    # --- 6. 處理自動清空垃圾桶 ---
    if AUTO_CLEAN:
        print("\n--- 自動清空垃圾桶設定已啟用 ---")
        if items_were_trashed or total_failed > 0: # 只有在移動過或有失敗時才嘗試清空
            empty_trash()
        else:
            print("ℹ️ 沒有項目被移至垃圾桶，無需清空。")
    else:
         if items_were_trashed: # 只有在移動過項目時才提示
             print("\nℹ️ 自動清空垃圾桶未啟用。您可以手動前往 Google Drive 清空垃圾桶。")

    print(f"\n🏁 ========== 程式執行完畢 ========== 🏁")


# 腳本說明：刪除指定資料夾下的個人檔案與空資料夾

## 用途

*   此腳本用於清理 Google Drive 中**指定的單一目標資料夾**及其所有子資料夾。
*   它會**遞迴地**檢查目標資料夾內的所有項目：
    *   如果項目是**檔案**，且**執行腳本者是該檔案的擁有者**，則將該檔案移至垃圾桶。
    *   如果項目是**子資料夾**，且**執行腳本者是該子資料夾的擁有者**：
        1.  腳本會先遞迴進入該子資料夾，處理其內部的檔案和更深層的子資料夾。
        2.  當處理完畢從子資料夾返回後，腳本會檢查該子資料夾**是否已變空**。
        3.  如果該子資料夾**已變空**，則將這個**空的子資料夾**移至垃圾桶。
*   **核心特性**：腳本嚴格遵守擁有者原則，**僅處理屬於執行腳本者本人擁有的檔案和（變空後的）子資料夾**。不屬於執行者的項目會被安全地跳過。
*   使用者可以選擇是否在處理完成後**自動清空垃圾桶**。
*   包含一個**Email 確認機制**，要求使用者輸入自己的 Email，以防止誤操作。

## 適用情境

*   清理特定的共享資料夾或專案資料夾，使用者只想移除自己建立或擁有的檔案，以及因移除檔案而變空的、自己擁有的資料夾結構，而不影響其他協作者的內容。
*   整理個人雲端硬碟中的某個特定資料夾，移除不再需要的個人檔案和空的子資料夾。
*   作為「依擁有者分類資料夾下檔案」腳本的**替代或補充**清理方案，直接在原始共享資料夾或其他指定位置進行清理，而不是在分類後的個人 Email 資料夾中操作。

## 權限要求

*   **執行者**：執行此腳本的 Google 帳號需要：
    *   對指定的「目標資料夾」擁有至少**檢視者 (Viewer)** 權限，以便能列出其內容。
    *   對「目標資料夾」及其內部要操作的子資料夾擁有**編輯者 (Editor)** 權限 (`capabilities.canEdit`)，因為將項目移至垃圾桶需要編輯權限。腳本會檢查頂層目標資料夾的編輯權限並發出警告（如果缺少）。
    *   **最重要**：對於目標資料夾內要被移至垃圾桶的**檔案**或**空的子資料夾**，執行者**必須是該項目的擁有者 (Owner)**。腳本會檢查此點，非擁有者的項目會被跳過。
    *   如果啟用了「自動清除垃圾桶」，則需要清空垃圾桶的權限（通常帳號擁有者都有）。

## 警告說明

*   **移至垃圾桶**：此腳本會將目標資料夾內**您所擁有**的檔案與**您所擁有且變空的子資料夾**移至垃圾桶。這不是立即永久刪除，但若後續清空垃圾桶，則無法復原。
*   **僅限擁有者**：再次強調，腳本**只會處理您擁有的項目**。目標資料夾內其他人擁有的檔案和非空資料夾會被保留。
*   **空資料夾判定**：只有在處理完一個資料夾內部、且該資料夾確實不再包含任何未被移入垃圾桶的項目時，腳本才會嘗試將這個（現在空的）資料夾移入垃圾桶（前提也是您擁有它）。
*   **Email 確認**：請務必在執行前，於「操作確認」欄位**正確輸入您當前登入 Colab/執行腳本的 Google 帳號 Email**。這是防止誤操作的重要保險步驟。輸入錯誤或留空將導致腳本中止。
*   **目標資料夾**：請確保「目標資料夾」的網址或 ID 輸入正確。錯誤的目標可能導致非預期的清理。
*   **自動清空垃圾桶**：如果啟用此選項，所有被移至垃圾桶的項目（包括本次操作和其他先前刪除的項目）將**立即被永久刪除**，無法復原。請謹慎啟用。
*   **權限影響**：如果在處理過程中缺少對某些子資料夾的編輯權限，相關項目可能無法被移至垃圾桶。
*   **API 限制**：處理包含大量項目的資料夾時，仍可能遇到 Google Drive API 的速率限制或超時。腳本包含延遲，但錯誤仍可能發生。